# AISA Capstone 2 Assignment

## Overview
This notebook provides an environment for you to build your intuition on the steps to take when developing a high quality Retrieval Augmented Generation (RAG) solution. 
RAG solutions retrieve data before calling the large language model (LLM) to generate an answer. 
The retrieved data is used to augment the prompt to the LLM by adding the relevant retrieved data in context. 
Any RAG solution is only as good as the quality of the data retrieval process, and this is the particular focus of this notebook, and the accompanying questions and tasks.

The RAG solution developed here is enabled by the Llamaindex framework. This is a popular framework in the industry for developing RAG and Agent based solutions. In addition to providing a core set of tools for orchestration of RAG and Agent workflows, there is broad integration with a variety of platforms for model inference (LLM, embedding, ...), and, importantly, tooling for solution evaluation.

## Prerequisites for running the notebook
- That you have granted access to the Bedrock models that you are going to use, in the region (**us-west-2**) where you are going to use Bedrock - 
[reference](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html)
- Your SageMakerExecutionRole has permissions to invoke Bedrock models - 
[reference](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-prereq.html)
- This notebook has been tested with SageMaker Notebook Instance running a `conda_python3` kernel
- The AWS region set for Amazon Bedrock use, needs to be in a region where the models being used are 1/ available, and 2/ enabled for use. This notebook was tested with Bedrock region `us-west-2`

## Implementation
This notebook uses llamaindex to define and execute the RAG solution. We will be using the following tools:

- **LLM (Large Language Model)**: e.g. Anthropic Claude Haiku available through Amazon Bedrock

  LLMs are used in the notebook for 1/ RAG response generation, to show the overall RAG workflow in actions, and 2/ for generating test questions on the indexed content (llamaindex nodes) for retrieval evaluation.
  
- **Text Embeddings Model**: e.g. Amazon Titan Embeddings available through Amazon Bedrock

  This embedding model is used to generate semantic vector representations of the content (llamaindex nodes) to be stored and the questions input to the RAG solution.
  
- **Document Loader**: SimpleDirectoryReader (Llamaindex)

  Before your chosen LLM can act on your data you need to load it. The way LlamaIndex does this is via data connectors, also called 'Reader'. Data connectors ingest data from different data sources and format the data into Document objects. A Document is a collection of data (currently text, and in future, images and audio) and metadata about that data.
  
  This implementation use SimpleDirectoryReader, which creates documents out of every file in a given directory. It can read a variety of formats including Markdown, PDFs, Word documents, and PowerPoint decks.

- **Vector Store**: VectorIndex (Llamaindex)

  In this notebook we are using this in-memory vector-store to store both the embeddings and the documents. In an enterprise context this could be replaced with a persistent store such as AWS OpenSearch, RDS Postgres with pgVector, ChromaDB, Pinecone or Weaviate.
  
  LlamaIndex abstracts the underlying vector database storage implementation with a VectorIndex class. This warps the Index, which is a data structure composed of Document objects, designed to enable querying by an LLM. The Index is designed to be complementary to your querying strategy.

----

## Setup

Install required Python modules for constructing the RAG solution.
You only need to run this once

In [1]:
%pip install \
    llama-index \
    llama-index-llms-bedrock \
    llama-index-embeddings-bedrock

Note: you may need to restart the kernel to use updated packages.


Download the default RAG test source data to our target source_docs directory. 
You only need to run this once.

In [2]:
source_docs_dir = './source_docs/'

The following creates the source_docs directory and downloads a document to that directory. The contents of this directory, 
initially the document that is downloaded here, will be used in the steps that follow.

After running this notebook in its entirity and reviewing its operation, delete this content and add your own content to the directory.

In [3]:
# Download and load data
!mkdir -p {source_docs_dir}
!wget --no-check-certificate 'https://www.buhurtinternational.com/_files/ugd/d219c5_5623c98ecdd142c8a7221c2d00cb621a.pdf' -O {source_docs_dir}'/buhurt_armor_requirement.pdf'
!wget --no-check-certificate 'https://www.buhurtinternational.com/_files/ugd/d219c5_afd49fe587524088b3136c910bb2ca51.pdf' -O {source_docs_dir}'/buhurt_weapon_requirement.pdf'

--2025-02-19 15:07:15--  https://www.buhurtinternational.com/_files/ugd/d219c5_5623c98ecdd142c8a7221c2d00cb621a.pdf
Resolving www.buhurtinternational.com (www.buhurtinternational.com)... 34.149.87.45
Connecting to www.buhurtinternational.com (www.buhurtinternational.com)|34.149.87.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12555242 (12M) [application/pdf]
Saving to: ‘./source_docs//buhurt_armor_requirement.pdf’

100%[======================================>] 12,555,242  61.7MB/s   in 0.2s   

2025-02-19 15:07:15 (61.7 MB/s) - ‘./source_docs//buhurt_armor_requirement.pdf’ saved [12555242/12555242]

--2025-02-19 15:07:15--  https://www.buhurtinternational.com/_files/ugd/d219c5_afd49fe587524088b3136c910bb2ca51.pdf
Resolving www.buhurtinternational.com (www.buhurtinternational.com)... 34.149.87.45
Connecting to www.buhurtinternational.com (www.buhurtinternational.com)|34.149.87.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 711

Import auxilliarty modules

In [4]:
import logging
import sys
import os
import pandas as pd
import boto3  # AWS SDK for Python

In [5]:
# This is required when running within a jupyter notebook, otherwise you will get errors when llamaindex modules run
import nest_asyncio

nest_asyncio.apply()

Import required Python modules for constructing and evaluating the RAG solution

In [6]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.bedrock import Bedrock

from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.text_splitter import TokenTextSplitter


from llama_index.core.evaluation import (
    DatasetGenerator,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Response,
)

## Configure the models that will be used for the RAG pipeline

**Note**: By default this notebook with use the `us-west-2` region. This region has support for the models used in this notebook. You should not need to change this setting.

In [7]:
# AWS_REGION = "us-west-2"
AWS_REGION = "us-east-1"  # this is an alternative setting to use if desired 

Define the set of Bedrock model IDs that we that we'll use when developing and testing our solution 

Establish a connection to the Amazon Bedrock service

In [8]:
boto3_bedrock = boto3.client("bedrock-runtime")

### Configure the target embeddings models for use with Llamaindex

In [9]:
titan_text_embeddings_multilingual_v1_id = "amazon.titan-embed-text-v1"
titan_text_embeddings_multilingual_v2_id = "amazon.titan-embed-text-v2:0"
cohere_text_embeddings_english_id = "cohere.embed-english-v3"
cohere_text_embeddings_multilingual_id = "cohere.embed-multilingual-v3"

Configure our chosen embeddings model for use with llama_index

In [10]:
titan_text_embeddings_v2 = BedrockEmbedding(model=titan_text_embeddings_multilingual_v2_id,region_name=AWS_REGION)
titan_text_embeddings_v1 = BedrockEmbedding(model=titan_text_embeddings_multilingual_v1_id,region_name=AWS_REGION)
cohere_text_embeddings_english = BedrockEmbedding(model=cohere_text_embeddings_english_id,region_name=AWS_REGION)
cohere_text_embeddings_multilingual= BedrockEmbedding(model=cohere_text_embeddings_english_id,region_name=AWS_REGION)

### Configure the target LLMs for use with Llamaindex

The following Mistral models produce good questions for evaluation. The Titan model produces questions of lesser quality 
and sometimes not in the format needed by the tools. 

**Note** Most Bedrock LLMs do note produce questions in a format that can be directly used for evaluation with the tooling as it is configured in this notebook.

In [11]:
instruct_mistral7b_id = "mistral.mistral-7b-instruct-v0:2"
instruct_mixtral8x7b_id="mistral.mixtral-8x7b-instruct-v0:1"
titan_text_express_id = "amazon.titan-text-express-v1"

In [12]:
# set the parameters to be applied when invoking the model
model_kwargs_llm = {
    "temperature": 0.5,
    "top_p": 0.9,
    "top_k": 200,
    "max_tokens": 4096
}

In [13]:
llm_mistral7b = Bedrock(model=instruct_mistral7b_id, client=boto3_bedrock, model_kwargs=model_kwargs_llm, region_name=AWS_REGION)
llm_mixtral8x7b = Bedrock(model=instruct_mixtral8x7b_id, client=boto3_bedrock, model_kwargs=model_kwargs_llm, region_name=AWS_REGION)
llm_titan_express = Bedrock(model=titan_text_express_id, client=boto3_bedrock, model_kwargs=model_kwargs_llm, region_name=AWS_REGION)

### Use the following cell to configure the embeddings model to use for the cells that follow

The embeddings model is a critical choice for the accuracy of your RAG solution.
Experiment with the options here to see which is best for your content.
If you want more, test with further alternatives. There are many that are readily supported by llama_index.

In [14]:
# KEY CELL 01

embed_model = titan_text_embeddings_v1
# embed_model = titan_text_embeddings_v2
# embed_model = cohere_text_embeddings_english
# embed_model = cohere_text_embeddings_multilingual

### Use the following cell to configure the LLM to use for the cells that follow
The LLM will be used for question generation and RAG answer generation in this notebook as it is currently configured.
The default value llm_mistral7b works well with the code and should be used if possible. 

In [15]:
llm_model = llm_mistral7b
# llm_model = llm_mixtral8x7b
# llm_model = llm_titan_express

In [16]:
# Set LlamaIndex default model settings to what was set in the cells above
Settings.embed_model = embed_model
Settings.llm = llm_model

## Read in the documents for adding to our data store

Read in the documents in the 'data/source_docs' directory into a structure ready for use by llama_index

In [17]:
reader = SimpleDirectoryReader(source_docs_dir)
documents = reader.load_data()

Quick check here to see that all of your documents were read. The count should match the number of pages in the documents in source_docs

In [18]:
len(documents)

25

## Create and run the document ingestion pipeline

The following cell defines two different document ingestion pipelines. 
If you have time, test using both of these, and create you own and test with that also.

In [19]:
# Define two transformation for the ingestion pipelines for initial experimentation

transformations_00=[
        TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=100),
        embed_model,
    ]

transformations_01=[
        SentenceSplitter(chunk_size=512, chunk_overlap=100),
        TitleExtractor(),
        embed_model,
    ]


### Use the following cell to configure the data ingestion pipeline for processing the source data

In [20]:
# KEY CELL 02

# create the pipeline with one of the transformation configurations defined above

pipeline = IngestionPipeline(transformations=transformations_00)
# pipeline = IngestionPipeline(transformations=transformations_01)


### Run the configured ingestion pipeline 

In [21]:
# run the pipeline
nodes = pipeline.run(documents=documents)
print(f"number of nodes: {len(nodes)}")

number of nodes: 26


This may make test analysis easier. It is none essential

In [22]:
# By default, the node ids are set to random uuids.
# To ensure same id's per run, we manually set them to consistent sequential numbers.
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

In [23]:
# validate that node has an embedding associated with it
for idx, node in enumerate(nodes):
    if node.id_ == "node_0":
        print(node.embedding)

[-0.240234375, 0.298828125, 0.3671875, -0.51171875, -0.2041015625, -0.0184326171875, -0.0126953125, 0.0004367828369140625, -0.1748046875, -0.2333984375, 0.07861328125, 0.23046875, -0.2421875, 0.2431640625, 0.06884765625, 0.12890625, 0.26953125, 0.66015625, -0.08935546875, 0.3359375, -0.0096435546875, 0.1923828125, -0.08203125, 0.5390625, -0.05712890625, -0.51171875, 0.1083984375, 0.004180908203125, -0.03759765625, 0.068359375, -0.671875, -0.134765625, -0.11181640625, -0.43359375, -0.032958984375, -0.08056640625, 0.3828125, 0.0361328125, 0.31640625, 0.0859375, 0.0341796875, 0.2890625, 0.1416015625, -0.8125, -0.306640625, -0.283203125, 0.115234375, 0.083984375, 0.2119140625, 0.2353515625, 0.1416015625, 0.380859375, 0.47265625, 0.1025390625, 0.057373046875, 0.515625, 0.263671875, 0.212890625, -0.2890625, -0.24609375, 0.271484375, 0.0125732421875, 0.033447265625, 0.466796875, 0.061767578125, 0.09619140625, 0.486328125, -0.365234375, 0.49609375, 0.373046875, 0.267578125, -0.13671875, 0.1865

## Create the VectorIndex 
This creates our vector database, in memory in this case,  using the nodes that were created in the previous step

In [24]:
vector_index = VectorStoreIndex(nodes=nodes)

## Test that we have a valid starting point for our evaluation
We run a quick system test with the defaul llama_index RAG workflow with a question that is relevant to our dataset

Instantiate a query engine object

In [25]:
query_engine = vector_index.as_query_engine()

Specify a question that has can be answered by the document(s) that have been ingested.

In [26]:
example_query="What are the historical consistency requirements of an armor?"

Run the default RAG pipeline with the example query. This should give a meaningful result. Don't worry if the answer is overly verbose, etc. We'll fix that later.

In [27]:
response = query_engine.query(example_query)
print(response)

 An armor must be derived from historical sources and align with authenticity rules documents. It must date between the 14th and 17th centuries, with reproductions of armors predating the 13th century being prohibited for safety considerations. Prohibited features include indications of modern materials or manufacturing techniques such as neon colors, obvious nylon cords, plastic ties, visible welded seams, and heat-induced discoloration. The armor must consist of pieces from the same style, with distinct styles defined as Western European, Slavic Influence, and Eastern Influence. Western European style includes countries such as Great Britain, Ireland, France, Portugal, Spain, Germany, Italy, Norway, Denmark, Sweden, Finland, Austria, Switzerland, Belgium, and the Netherlands. Slavic Influence includes countries such as the Czech Republic, Romania, Hungary, Poland, Slovakia, Slovenia, Croatia, Latvia, Estonia, Moldova, Serbia, Ukraine, Russia, and Belarus.


----

# Evaluate the retrieval accuracy of the VectorIndex

## Create a set of question and node (context) pairs to drive the tests that follow
This uses the llm that give the methods and the document data stored in the nodes (created during document ingestion)

This will make many calls to the specified LLM (num_questions_per_chunk * number of nodes). This will likely be throttled by Bedrock. The llama_index API will work through the throttling except in extreme cases.

In [28]:
%%time
qa_dataset = generate_question_context_pairs(nodes, num_questions_per_chunk=1)

 35%|███▍      | 9/26 [01:09<03:21, 11.88s/it]Retrying llama_index.llms.bedrock.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again..
Retrying llama_index.llms.bedrock.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again..
 54%|█████▍    | 14/26 [02:02<01:44,  8.68s/it]Retrying llama_index.llms.bedrock.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ThrottlingException: An error occurred (ThrottlingException) wh

CPU times: user 425 ms, sys: 23.6 ms, total: 449 ms
Wall time: 4min 42s


Take a look at the sample queries generated. This should show a meaningful questions related to your document content.

In [29]:
for item in list(qa_dataset.queries.items()):
    print(item[1])

Based on the context information given, here is a question that could be asked for an upcoming quiz or examination:
What are the historical consistency requirements for armor equipment according to the given document?
In what categories does the document outline the use of armors, shields, and weapons in combat?
Which historical period does the 14th century Western European style of armor belong to?
Which email address should competitors contact to obtain validation for deviations from the specified armor styles as outlined in the technical requirements document?
What are the two main responsibilities of every competitor regarding their equipment in full contact fights according to the given context?
What are the minimum requirements for a helmet as stated in the context document?
What material is recommended for the dome and visor of a helmet according to the given document?
What is the minimum recommended thickness for mild steel plates used in armor?
What type of gauntlets cannot be

## Instantiate a retriever against the index for testing



### Set the number of items to return from the Retriever
This is a trade-off item, more returned content is not always better. Consider how this may impact your pipeline and evaluation results and experiment with it.

In [30]:
# KEY CELL: 03

number_of_items_to_return = 2
# number_of_items_to_return = 3
# number_of_items_to_return = 4

In [31]:
retriever = vector_index.as_retriever(similarity_top_k=number_of_items_to_return)

Run a quick system test on the retriever and check that the output nodes look reasonable

In [32]:
retrieved_nodes = retriever.retrieve(example_query)
print(retrieved_nodes)

[NodeWithScore(node=TextNode(id_='node_3', embedding=None, metadata={'page_label': '4', 'file_name': 'buhurt_armor_requirement.pdf', 'file_path': '/home/ec2-user/SageMaker/elvtr-ai-solution-architect/class-11/source_docs/buhurt_armor_requirement.pdf', 'file_type': 'application/pdf', 'file_size': 12555242, 'creation_date': '2025-02-19', 'last_modified_date': '2024-12-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5d80a8a0-7d8b-4e57-9470-9a4255bd619e', node_type='4', metadata={'page_label': '4', 'file_name': 'buhurt_armor_requirement.pdf', 'file_path': '/home/ec2-user/SageMaker/elvtr-ai-solution-architect/class-11/source_docs/buhurt_armor_requirement.pdf', 'file_type': 'application/pdf', 'file_size': 1

## Evaluate the Quality of Retrieval from the VectorIndex

Instantiate a RetrieverEvaluator with the metrics that we want to review

In [33]:
metrics = ["hit_rate", "mrr", "precision", "recall"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(metrics, retriever=retriever)

In [34]:
# Evaluate on a single query
# The output is verbose, but may be useful for looking at specific results

query_id = 1  # change this to math the query id of interest

sample_id, sample_query = list(qa_dataset.queries.items())[query_id]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Query: What are the historical consistency requirements for armor equipment according to the given document?
Metrics: {'hit_rate': 0.0, 'mrr': 0.0, 'precision': 0.0, 'recall': 0.0}



In [35]:
# to see detail on which nodes were returned, etc, we can look at the whole returned object
eval_result

RetrievalEvalResult(query='What are the historical consistency requirements for armor equipment according to the given document?', expected_ids=['node_1'], expected_texts=None, retrieved_ids=['node_3', 'node_15'], retrieved_texts=['2. Historical ConsistencyRequirements\n2.1.Equipmentfromhistoricalsources● Onlyarmorsderivedfromhistorical sourcesarepermittedforuse.● Armorsmust alignwithAuthenticityRulesdocuments.\n2.2.Datesofsources● Armorsmust alignwithsourcesdatingbetweenthe14th(1300)and17th(1600)centuries.● For safety considerations, reproductions of armors predating the 13th century areprohibited.\n2.3.Prohibitedfeatures● Prohibited features encompass evident indications of modern materials or manufacturingtechniques including: neoncolors, obvious nyloncords, plastic ties, visibleweldedseams,heat-induceddiscoloration, modernfootwearandothervisiblemodernequipment.\n2.4.ConsistencyinEquipmentArmors, shields and weapons must consist of pieces fromthesamestyle. Distinct styles aredefined

In [36]:
### Run evaluation on the entire test dataset (autogenerated above)
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [37]:
def display_results(name, eval_results):
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)
    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()
    precision = full_df["precision"].mean()
    recall = full_df["recall"].mean()

    metric_df = pd.DataFrame({"retrievers": [name],
                              "hit_rate": [hit_rate], "mrr": [mrr],
                              "precision": [precision], "recall": [recall],
                             })
    return metric_df, full_df


### Top-level Evaluation Results

In [38]:
summary, detail = display_results(f"top-{number_of_items_to_return} eval", eval_results)
summary

,retrievers,hit_rate,mrr,precision,recall
0,top-2 eval,0.730769,0.596154,0.365385,0.730769


In [39]:
# Optionally, look at the detailed, question by question metrics:
detail

,hit_rate,mrr,precision,recall
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,1.0,1.0,0.5,1.0
4,1.0,1.0,0.5,1.0
5,1.0,0.5,0.5,1.0
6,1.0,0.5,0.5,1.0
7,1.0,1.0,0.5,1.0
8,1.0,1.0,0.5,1.0
9,1.0,1.0,0.5,1.0


----

# Assignment Instructions

**01** Run the notebook in full, with the default, provided, document set (one document in .txt file).

Copy the Top-level Evaluation Results (from the cell a little way above this one) into this cell.

Save the notebook and rename is as `capstone-02-01-first-run.ipynb`. Download the notebook for assignment submission.

**Answer**

| Metric | Value |
|--------|--------|
| Retriever Hit Rate | 0.636364 |
| MRR | 0.534091 |
| Precision | 0.318182 |
| Recall | 0.636364 |


**02** Replace the default document with one of more of your own documents and re-run all the cells.

- First make a copy of the first notebook and paste it into the same directory.
- Delete the original content on source_docs
- Delete the cell in the notebook that will download the original content again to source_docs
- Upload your content to source_docs
- Initially test with a small set - 20 to 40 nodes aka document chunks - to save you time as you experiment. This may mean that you delete some of your content, to reduce its size. 
- Run the whole notebook with your small set of  test documents and observe the results.

Wrapping up:
- Copy the Top-level Evaluation Results (from the cell a little way above this one) into this cell.
- The top-level results returned are your `baseline` results. As you experiment with different configurations, in the following assignment tasks, you will see the route to improved accuracy from this baseline.
- Describe your test dataset in this cell. How many documents, how many chunks, what is the topic of the content, what is the language.

This completes this step.

- Save and rename the notebook as `capstone-02-02-my-data-baseline.ipynb`. Download the notebook for assignment submission.

**Answer**

| Metric | Value |
|--------|--------|
| Retriever Hit Rate | 0.730769 |
| MRR | 0.596154 |
| Precision | 0.365385 |
| Recall | 0.730769 |

The test dataset is 2 PDF outlining armor and weapon requirements for medieval sport combat (also known as buhurt). I updated the code to retrieve the documents directly from the source using `wget`.

The 2 PDF combined represents **25 pages**, which generated **26 nodes** using `chunk_size` of **512**.

The PDFs are in **English**.


**03** Using your data experiment with different embedding models

- First make a copy of the previous notebook,`capstone-02-02-my-data-baseline.ipynb`, and paste it into the same directory.
- Change which embedding model is going to be used from the original setting. 
There are three other options prepared in this notebook, see `KEY CELL 01`. 
- Run all the following cells in the notebook and note the in accuracy metrics for the selected model.
- Repeat with one or more of the embeddings models, noting the accuracy metrics for the selected model, each time.
Note: Changing the embeddings model will change the accuracy of the retriever. Exactly how much will depend on your content.

Wrapping up:
- For each embeddings model that you experiment with, copy the Top-level Evaluation Results (from the cell a little way above this one) into this cell, along with an indication as to which embeddings model was being used.

- Briefly describe your results in this cell. Which model was best, did you see a major improvement in the metrics, can you suggest a reason for why the best embeddings model is performing better than the worst.

This completes this step.


- Save and rename the notebook as `capstone-02-03-my-data-embeddings.ipynb`. Download the notebook for assignment submission.

**04** Using your data experiment with different ingestion pipelines

- First make a copy of the previous notebook,`capstone-02-03-my-data-embeddings.ipynb`, and paste it into the same directory.
- Make sure the the notebook is configured to use the best performing embeddings model.
- Change which pipeline is going to be used from the original setting. 
There is one other options prepared in this notebook, see `KEY CELL 02`.
- Run all the following cells in the notebook and note the in accuracy metrics for the selected pipeline.
- Optionally, create your own pipeline and experiment with that to further improve the  accuracy metrics of your solution.
Note: Changing the ingestion should change the accuracy of the retriever. Exactly how much will depend on your content. 
The two example pipelines in this notebook may not make much of a difference for your content. 
If you have time, you will learn most by experimenting with creating you own and seeing the change in the metrics.

Wrapping up:
- For each pipeline that you experiment with, copy the Top-level Evaluation Results (from the cell a little way above this one) into this cell, along with the definition of the pipeline being used.

- Briefly describe your results in this cell. Which pipeline was best, did you see a major improvement in the metrics, can you suggest a reason for why the best pipeline is performing better than the worst.

This completes this step.


- Save and rename the notebook as `capstone-02-04-my-data-pipeline.ipynb`. Download the notebook for assignment submission.

**05** Using your data experiment with different values of k

- First make a copy of the previous notebook,`capstone-02-04-my-data-pipeline.ipynb`, and paste it into the same directory.
- Make sure the the notebook is configured to use the best performing pipeline and embeddings model.
- Change the value of *k* that is going to be used from the original setting. 
There are two other options prepared in this notebook, see `KEY CELL 03`.
- Run all the cells that follow below in the notebook and note the in accuracy metrics for the selected value of *k*. 
*Note*: with this setting, you do not need to re-run the cells above the cell where you make the update.


Wrapping up:
- For each value of *k* set for the `retriever`, run the evaluation and copy the Top-level Evaluation Results into this cell, along with noting the value of *k* being used.

- Briefly describe your results in this cell. Which value of *k* gave the best results, did you see a major improvement in the metrics, can you suggest a reason for why the best pipeline is performing better than the worst.

This completes this step.


- Save and rename the notebook as `capstone-02-05-my-data-pipeline-with-k.ipynb`. Download the notebook for assignment submission.

**06** Summarize

- Breifly summarize, in four paragraphs what your learned, regarding the following topics:
    - The configuration of the retriever for your content
    - The process of experimentating with different settings and evaluating the results
    - Which evaluation metric was most useful and why
    - What might you do next, if you had a 40 hours or more to work on this, to further improve the quality of the retriever
- Save your summary in this cell.
    
This completes this step.


- Save and rename the notebook as `capstone-02-06-summary.ipynb`. Download the notebook for assignment submission.
    


Post each of the notebooks, individually, to submit your assignment. Do not zip the set of notebooks, as that makes it harder for the grading process. 

## The following assignment tasks are completely optional 
The follow tasks are intended for students who want to dive deeper. They are more open ended and require changing and augmenting the code share above.

**Optional Task 1** Text Chunking

Experiment further with advanced chunking options and see if you can further improve the accuracy of the your Retriever (by having the better pre-processed data).

Good candidates to look are [Semantic chunking](https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/), and
[Semantic double merging chunking](https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_double_merging_chunking/).

**Optional Task 2** Explore other Transformations

Using the following [document](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/transformations/) as your starting point, consider which transformations would be most applicable to your set of documents and experiment and see the impact of those changes.

There are lots of [transformations](https://docs.llamaindex.ai/en/v0.9.48/module_guides/loading/ingestion_pipeline/transformations.html) to consider including 
[text splitters](https://docs.llamaindex.ai/en/v0.9.48/module_guides/loading/node_parsers/modules.html#text-splitters),
[node parsers](https://docs.llamaindex.ai/en/v0.9.48/module_guides/loading/node_parsers/modules.html), 
and [metadata extractors](https://docs.llamaindex.ai/en/v0.9.48/module_guides/loading/documents_and_nodes/usage_metadata_extractor.html).



**Optional Task 3** Embeddings Model

Experiment further with other state of the art embeddings model and see if you can further improve the accuracy of the your Retriever (by having the better pre-processed data).

The embeddings models developed by [Voyage AI](https://www.voyageai.com) are some of the best in the industry. They also provide generous free-tier use of the embeddings models (as a service). Signing-up to get a developer account is a fairly light-weight process. 

The benefit that you'll get is 1/ seeing how to use a whole new model family for embeddings with Llamaindex, 2/ a deeper knowledge of your embeddings options, and 3/ perhaps, a more accurate Retriever for your solution.

If you prefer to try other embeddings models, such as those provided by OpenAI, that's also well worth exploring.